# Graded Assignment 2 - Air Quality and Weather in the Netherlands


# Introduction to This Template Notebook

* This is a **group** notebook.
* Make sure you work in a **copy** of `...-template.ipynb`,
**renamed** to `...-yourIDnrs.ipynb`,
where `yourIDnrs` is the TU/e identification numbers of the members of the group.

<div class="alert alert-danger" role="danger">
<h3>Integrity</h3>
<ul>
    <li>In this course you must act according to the rules of the TU/e code of scientific conduct.</li>
    <li>This exercise or graded assignment is to be executed by the members of the group independently from other people.</li>
    <li>You must not copy from the Internet, your friends, books... If you represent other people's work as your own, then that constitutes fraud and will be reported to the Examination Committee.</li>
    <li>Making your work available to others (complicity) also constitutes fraud.</li>
</ul>
</div>

You are expected to work with Python code and Markdown in this notebook.

Proceed in this notebook as follows:
* **Read** the assignment (separate PDF).
* **Write** your decisions/solutions/interpretations in the appropriate sections.
* **Run** _all_ code cells (also the ones _without_ your code),
    _in linear order_ from the first code cell.

**Personalize your notebook**:
1. Copy the following line of code:

  ```python
  AUTHOR_ID_NRS = ['1234567', '2234567', '3234567', '4234567']
  ```
1. Paste them between the marker lines in the next code cell.
1. Fill in the _identification numbers_ of all members of the group as a list of strings between the `Author` markers.
1. Run the code cell by putting the cursor there and typing **Control-Enter**.


In [68]:
#// BEGIN_TODO [Author] Name, Id.nr., Date, as strings (1 point)

AUTHOR_ID_NRS =[]

#// END_TODO [Author]

AUTHOR_ID_NRS

[]

## Table of Contents

- [Preparation](#Preparation)
    - [Load the libraries](#Load-the-libraries)
- [Part 1a. Hypothesis selection](#Part-1a:-Hypothesis-selection)
- [Part 1b. Hypothesis refinement](#Part-1b:-Hypothesis-refinement)
- [Part 2. Queries and data cleaning](#Part-2:-Queries-and-data-cleaning)
- [Part 3. Hypothesis testing and interpretation](#Part-3.-Hypothesis-testing-and-interpretation)
- [Part 4. Pitching results](#Part-4.-Pitching-results)

## Preparation
### Load the libraries

In [69]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression         # for linear regression
from sklearn.cluster import KMeans                        # for clustering
from sklearn.tree import DecisionTreeClassifier           # for decision tree mining
from sklearn.metrics import mean_absolute_error, confusion_matrix
from sklearn.model_selection import train_test_split
from treeviz import tree_print                            # to print decision tree

import scipy.stats as stats                               # to compute z-scores
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d
import sqlite3                                            # to interact with the database
import statsmodels.api as sm
from statsmodels.distributions.empirical_distribution import ECDF

%matplotlib inline                                 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns                                     # also improves the look of plots
sns.set()
plt.rcParams['figure.figsize'] = 10, 5                    # default hor./vert. size of plots, in inches
plt.rcParams['lines.markeredgewidth'] = 1                 # to fix issue with seaborn box plots; needed after import seaborn

## Part 1a: Hypothesis selection

In [70]:
hypothesis = 'During the winter/cold months we have a higher concentration of air pollutants'
hypothesis

'During the winter/cold months we have a higher concentration of air pollutants'

## Part 1b: Hypothesis refinement

In [71]:
# Use this cell as you like, and add more cells as needed.

## Part 2: Queries and data cleaning

In [72]:
factors = ['temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed']
compounds = ['no_x','o_3']

In [73]:
def process_query(query, params = None) -> pd.DataFrame: # Tim
    """
    Returns dataframe based on given query
    :param query: given query
    :param params: possible params to pass into query
    :returns: dataframe
    """
    with sqlite3.connect(r'./datasets/aqw.db') as conn:
        return_df = pd.read_sql_query(query, conn, params=params)
    return return_df

def get_weather_stations() -> pd.DataFrame: # Tim
    """
    Returns dataframe of all weather stations
    :returns: dataframe
    """
    query = '''
    SELECT code, name, latitude, longitude 
    FROM weather_stations'''
    
    return process_query(query)
    
def get_aq_stations() -> pd.DataFrame: # Tim
    """
    Returns dataframe of all air quality stations
    :returns: dataframe
    """
    query = '''
    SELECT code, name, latitude, longitude 
    FROM air_quality_stations'''
    
    return process_query(query)

def get_correct_stations(df_stations) -> pd.DataFrame: # Tim
    """
    Filters out stations that are not needed in future
    :param df_stations: dataframe of all stations
    :returns: dataframe of needed stations
    """
    df_stations['type'] = 'none'
    df_stations.loc[df_stations['longitude']<4, 'type'] = 'west'
    df_stations.loc[(df_stations['latitude']>53) & (df_stations['longitude']>6), 'type'] = 'north'
    df_stations.loc[df_stations['latitude']<51, 'type'] = 'south'
    return df_stations[df_stations['type']!='none']

def get_weather_data(codes, factors) -> pd.DataFrame: # Tim
    """
    Returns weather measurements of given factors from given stations list
    :param codes: list of stations by codes
    :param factors: list of factors
    :returns: dataframe of measurements
    """
    sql_factors = ', '.join(factors)
    sql_codes = ', '.join(str(code) for code in codes)
    query = f'''
    SELECT code, name, datetime, {sql_factors} 
    FROM weather_stations, weather_data
    WHERE weather_stations.code = weather_data.station_code AND weather_data.station_code IN ({sql_codes})
    '''
    
    return process_query(query)

def get_aq_data(codes, compounds) -> pd.DataFrame: # Tim
    """
    Returns air quality measurements of given compounds from given stations list
    :param codes: list of stations by codes
    :param factors: list of compounds
    :returns: dataframe of measurements
    """
    sql_compounds = ', '.join(compounds)
    sql_codes = ', '.join(f"'{code}'" for code in codes)
    query = f'''
    SELECT code, name, datetime, {sql_compounds} 
    FROM air_quality_stations, air_quality_data
    WHERE air_quality_stations.code = air_quality_data.station_code AND air_quality_data.station_code IN ({sql_codes})
    '''
    
    return(process_query(query))

def get_divided_dates(df_stations) -> pd.DataFrame: # Ansat
    '''
    Firstly, converts datetime in dataframe to datetime from object
    Then adds to a given dataframe year, month, day_of_year, week, and hour columns, 
    so we would have more control over data
    :param df_data: given data frame
    :returns: dataframe with divided datetime column
    '''
    df_stations['datetime'] = pd.to_datetime(df_stations['datetime'])
    df_stations['year'] = df_stations['datetime'].dt.year
    df_stations['month'] = df_stations['datetime'].dt.month
    df_stations['day_of_year'] = df_stations['datetime'].dt.dayofyear
    df_stations['week'] = df_stations['day_of_year'] // 7 + 1
    df_stations['hour'] = df_stations['datetime'].dt.hour
    df_stations = df_stations.set_index('datetime') 
    return df_stations

def get_aq_station_type(df_aq_station) -> pd.DataFrame: # Ansat
    '''
    Gives categories to the air quality stations so we would understand nearby which source they are located
    :params: Given dataframe of weather data
    :returns: dataframe with categories of weather stations
    '''
    df_aq_station['station_type'] = 'NaN'
    df_aq_station.loc[df_aq_station['name'] == ('Philippine-Stelleweg' or 'Wijnandsrade-Opfergeltstraat'), 
                      'station_type'] = 'background'
    df_aq_station.loc[df_aq_station['name'] == ('Sluiskil-Stroodorpestraat' or 'Geleen-Asterstraat' or 
                                                'Geleen-Vouershof'), 'station_type'] = 'industrial'
    df_aq_station.loc[df_aq_station['name'] == ('Groningen-Europaweg' or 'Heerlen-Looierstraat' or 
                                                'Maastricht-A2 Nassaulaan'), 'station_type'] = 'street'
    df_aq_station.loc[df_aq_station['name'] == ('Groningen-Nijensteinheerd' or 'Heerlen-Jamboreepad'), 
                      'station_type'] = 'city'
    return df_aq_station

def construct_dfs(factors, compounds) -> list[pd.DataFrame]: # Tim
    """
    Constructs 3 dataframes with weather meauseremnts and 3 dataframes with corresponding air quality meauserements
    based on given weather factors and compounds
    :param factors: given weather factors
    :param compounds: given compounds
    :returns: list of 6 dataframes
    """
    df_ws = get_correct_stations(get_weather_stations())
    df_aqs = get_correct_stations(get_aq_stations())
    
    df_wd_west=get_divided_dates(get_weather_data(df_ws[df_ws['type'] == 'west']['code'], factors))
    df_aqd_west=get_divided_dates(get_aq_data(df_aqs[df_aqs['type'] == 'west']['code'], compounds))
    df_aqd_west=get_aq_station_type(df_aqd_west)
    
    df_wd_north=get_divided_dates(get_weather_data(df_ws[df_ws['type'] == 'north']['code'], factors))
    df_aqd_north=get_divided_dates(get_aq_data(df_aqs[df_aqs['type'] == 'north']['code'], compounds))
    df_aqd_north=get_aq_station_type(df_aqd_north)
    
    df_wd_south=get_divided_dates(get_weather_data(df_ws[df_ws['type'] == 'south']['code'], factors))
    df_aqd_south=get_divided_dates(get_aq_data(df_aqs[df_aqs['type'] == 'south']['code'], compounds))
    df_aqd_south=get_aq_station_type(df_aqd_south)
    
    
    return df_wd_west, df_wd_north, df_wd_south, df_aqd_west, df_aqd_north, df_aqd_south



In [74]:
df_wd_west, df_wd_north, df_wd_south, df_aqd_west, df_aqd_north, df_aqd_south = construct_dfs(factors, compounds)

In [75]:
print(df_aqd_north.info())
for compound in compounds:
    print(f"North station:  \n Name: {compound}, \nNumber of non-null values: {len(df_aqd_north[df_aqd_north[compound].isnull()==False])}")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 175342 entries, 2012-01-01 01:00:00+01:00 to 2021-12-31 23:00:00+01:00
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   code          175342 non-null  object 
 1   name          175342 non-null  object 
 2   no_x          170206 non-null  float64
 3   o_3           85750 non-null   float64
 4   year          175342 non-null  int64  
 5   month         175342 non-null  int64  
 6   day_of_year   175342 non-null  int64  
 7   week          175342 non-null  int64  
 8   hour          175342 non-null  int64  
 9   station_type  175342 non-null  object 
dtypes: float64(2), int64(5), object(3)
memory usage: 14.7+ MB
None
North station:  
 Name: no_x, 
Number of non-null values: 170206
North station:  
 Name: o_3, 
Number of non-null values: 85750


In [76]:
print(df_aqd_south.info())
for compound in compounds:
    print(f"South station:  \n Name: {compound}, \nNumber of non-null values: {len(df_aqd_south[df_aqd_south[compound].isnull()==False])}")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 543573 entries, 2012-01-01 01:00:00+01:00 to 2021-01-01 00:00:00+01:00
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   code          543573 non-null  object 
 1   name          543573 non-null  object 
 2   no_x          381784 non-null  float64
 3   o_3           170506 non-null  float64
 4   year          543573 non-null  int64  
 5   month         543573 non-null  int64  
 6   day_of_year   543573 non-null  int64  
 7   week          543573 non-null  int64  
 8   hour          543573 non-null  int64  
 9   station_type  543573 non-null  object 
dtypes: float64(2), int64(5), object(3)
memory usage: 45.6+ MB
None
South station:  
 Name: no_x, 
Number of non-null values: 381784
South station:  
 Name: o_3, 
Number of non-null values: 170506


In [77]:
print(df_aqd_west.info())
for compound in compounds:
    print(f"West station:  \n Name: {compound}, \nNumber of non-null values: {len(df_aqd_west[df_aqd_west[compound].isnull()==False])}")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 149062 entries, 2012-01-01 01:00:00+01:00 to 2015-01-01 00:00:00+01:00
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   code          149062 non-null  object 
 1   name          149062 non-null  object 
 2   no_x          93996 non-null   float64
 3   o_3           82567 non-null   float64
 4   year          149062 non-null  int64  
 5   month         149062 non-null  int64  
 6   day_of_year   149062 non-null  int64  
 7   week          149062 non-null  int64  
 8   hour          149062 non-null  int64  
 9   station_type  149062 non-null  object 
dtypes: float64(2), int64(5), object(3)
memory usage: 12.5+ MB
None
West station:  
 Name: no_x, 
Number of non-null values: 93996
West station:  
 Name: o_3, 
Number of non-null values: 82567


### Data about north stations

In [78]:
df_wd_north

,code,name,temperature,snow,thunder,sunshine_duration,wind_speed,year,month,day_of_year,week,hour
datetime,,,,,,,,,,,,
2012-01-01 00:00:00+01:00,280,Eelde,10.2,0.0,0.0,0.0,8.0,2012,1,1,1,0
2012-01-01 01:00:00+01:00,280,Eelde,10.0,0.0,0.0,0.0,6.0,2012,1,1,1,1
2012-01-01 02:00:00+01:00,280,Eelde,10.1,0.0,0.0,0.0,7.0,2012,1,1,1,2
2012-01-01 03:00:00+01:00,280,Eelde,10.4,0.0,0.0,0.0,8.0,2012,1,1,1,3
2012-01-01 04:00:00+01:00,280,Eelde,10.6,0.0,0.0,0.0,7.0,2012,1,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00+01:00,280,Eelde,10.5,0.0,0.0,0.0,6.0,2021,12,365,53,19
2021-12-31 20:00:00+01:00,280,Eelde,10.9,0.0,0.0,0.0,6.0,2021,12,365,53,20
2021-12-31 21:00:00+01:00,280,Eelde,11.3,0.0,0.0,0.0,6.0,2021,12,365,53,21


In [79]:
df_aqd_north

,code,name,no_x,o_3,year,month,day_of_year,week,hour,station_type
datetime,,,,,,,,,,
2012-01-01 01:00:00+01:00,NL10937,Groningen-Europaweg,113.320,NaN,2012,1,1,1,1,street
2012-01-01 02:00:00+01:00,NL10937,Groningen-Europaweg,73.990,NaN,2012,1,1,1,2,street
2012-01-01 03:00:00+01:00,NL10937,Groningen-Europaweg,48.910,NaN,2012,1,1,1,3,street
2012-01-01 04:00:00+01:00,NL10937,Groningen-Europaweg,42.810,NaN,2012,1,1,1,4,street
2012-01-01 05:00:00+01:00,NL10937,Groningen-Europaweg,29.680,NaN,2012,1,1,1,5,street
...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00+01:00,NL10938,Groningen-Nijensteinheerd,72.085,41.32,2021,12,365,53,19,city
2021-12-31 20:00:00+01:00,NL10938,Groningen-Nijensteinheerd,79.092,37.87,2021,12,365,53,20,city
2021-12-31 21:00:00+01:00,NL10938,Groningen-Nijensteinheerd,53.019,38.08,2021,12,365,53,21,city


### Data about south stations

In [80]:
df_wd_south

,code,name,temperature,snow,thunder,sunshine_duration,wind_speed,year,month,day_of_year,week,hour
datetime,,,,,,,,,,,,
2012-01-01 00:00:00+01:00,380,Maastricht,10.7,0,0,0.0,7.0,2012,1,1,1,0
2012-01-01 01:00:00+01:00,380,Maastricht,10.8,0,0,0.0,7.0,2012,1,1,1,1
2012-01-01 02:00:00+01:00,380,Maastricht,10.6,0,0,0.0,7.0,2012,1,1,1,2
2012-01-01 03:00:00+01:00,380,Maastricht,10.5,0,0,0.0,7.0,2012,1,1,1,3
2012-01-01 04:00:00+01:00,380,Maastricht,10.6,0,0,0.0,7.0,2012,1,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00+01:00,380,Maastricht,12.7,0,0,0.0,5.0,2021,12,365,53,19
2021-12-31 20:00:00+01:00,380,Maastricht,12.8,0,0,0.0,4.0,2021,12,365,53,20
2021-12-31 21:00:00+01:00,380,Maastricht,12.7,0,0,0.0,3.0,2021,12,365,53,21


In [81]:
df_aqd_south

,code,name,no_x,o_3,year,month,day_of_year,week,hour,station_type
datetime,,,,,,,,,,
2012-01-01 01:00:00+01:00,NL10136,Heerlen-Looierstraat,NaN,NaN,2012,1,1,1,1,NaN
2012-01-01 02:00:00+01:00,NL10136,Heerlen-Looierstraat,15.89,NaN,2012,1,1,1,2,NaN
2012-01-01 03:00:00+01:00,NL10136,Heerlen-Looierstraat,13.42,NaN,2012,1,1,1,3,NaN
2012-01-01 04:00:00+01:00,NL10136,Heerlen-Looierstraat,10.46,NaN,2012,1,1,1,4,NaN
2012-01-01 05:00:00+01:00,NL10136,Heerlen-Looierstraat,9.91,NaN,2012,1,1,1,5,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-12-31 20:00:00+01:00,NL50009,Maastricht-A2 Kasteel Hillenraadweg,NaN,NaN,2020,12,366,53,20,NaN
2020-12-31 21:00:00+01:00,NL50009,Maastricht-A2 Kasteel Hillenraadweg,NaN,NaN,2020,12,366,53,21,NaN
2020-12-31 22:00:00+01:00,NL50009,Maastricht-A2 Kasteel Hillenraadweg,NaN,NaN,2020,12,366,53,22,NaN


### Data about western stations

In [82]:
df_aqd_west

,code,name,no_x,o_3,year,month,day_of_year,week,hour,station_type
datetime,,,,,,,,,,
2012-01-01 01:00:00+01:00,NL10318,Philippine-Stelleweg,4.49,52.88,2012,1,1,1,1,background
2012-01-01 02:00:00+01:00,NL10318,Philippine-Stelleweg,3.77,55.04,2012,1,1,1,2,background
2012-01-01 03:00:00+01:00,NL10318,Philippine-Stelleweg,3.95,55.66,2012,1,1,1,3,background
2012-01-01 04:00:00+01:00,NL10318,Philippine-Stelleweg,4.74,54.81,2012,1,1,1,4,background
2012-01-01 05:00:00+01:00,NL10318,Philippine-Stelleweg,4.14,56.58,2012,1,1,1,5,background
...,...,...,...,...,...,...,...,...,...,...
2014-12-31 20:00:00+01:00,NL10319,Nieuwdorp-Coudorp,NaN,NaN,2014,12,365,53,20,NaN
2014-12-31 21:00:00+01:00,NL10319,Nieuwdorp-Coudorp,NaN,NaN,2014,12,365,53,21,NaN
2014-12-31 22:00:00+01:00,NL10319,Nieuwdorp-Coudorp,NaN,NaN,2014,12,365,53,22,NaN


In [83]:
df_wd_north

,code,name,temperature,snow,thunder,sunshine_duration,wind_speed,year,month,day_of_year,week,hour
datetime,,,,,,,,,,,,
2012-01-01 00:00:00+01:00,280,Eelde,10.2,0.0,0.0,0.0,8.0,2012,1,1,1,0
2012-01-01 01:00:00+01:00,280,Eelde,10.0,0.0,0.0,0.0,6.0,2012,1,1,1,1
2012-01-01 02:00:00+01:00,280,Eelde,10.1,0.0,0.0,0.0,7.0,2012,1,1,1,2
2012-01-01 03:00:00+01:00,280,Eelde,10.4,0.0,0.0,0.0,8.0,2012,1,1,1,3
2012-01-01 04:00:00+01:00,280,Eelde,10.6,0.0,0.0,0.0,7.0,2012,1,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00+01:00,280,Eelde,10.5,0.0,0.0,0.0,6.0,2021,12,365,53,19
2021-12-31 20:00:00+01:00,280,Eelde,10.9,0.0,0.0,0.0,6.0,2021,12,365,53,20
2021-12-31 21:00:00+01:00,280,Eelde,11.3,0.0,0.0,0.0,6.0,2021,12,365,53,21


### Information about empty columns in tables (possibly compounds which we can drop)

In [84]:
df_wd_west = df_wd_west.dropna(how='all', axis = 1)
df_wd_north = df_wd_north.dropna(how='all', axis = 1)
df_wd_sotuh = df_wd_south.dropna(how='all', axis = 1)

df_aqd_west = df_aqd_west.dropna(how='all', axis = 1)
df_aqd_north = df_aqd_north.dropna(how='all', axis = 1)
df_aqd_south = df_aqd_south.dropna(how='all', axis = 1)

north_compounds = df_aqd_north.columns.values.tolist()[2:-6]
south_compounds = df_aqd_south.columns.values.tolist()[2:-6]
west_compounds = df_aqd_west.columns.values.tolist()[2:-6]

### Finding missing/invalid air quality stations data and dropping irrelevant rows

In [85]:
df_aqd_north = df_aqd_north.dropna(subset = north_compounds, how='all')
df_aqd_south = df_aqd_south.dropna(subset = south_compounds, how='all')
df_aqd_west = df_aqd_west.dropna(subset = west_compounds, how='all')

### Cleaning invalid data in weather stations

In [86]:
factors

['temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed']

In [87]:
print(len(df_wd_south))
for factor in factors: 
    print(f"Number of missing rows with {factor} columns: {len(df_wd_south[df_wd_south[factor].isnull()])}")

87672
Number of missing rows with temperature columns: 0
Number of missing rows with snow columns: 0
Number of missing rows with thunder columns: 0
Number of missing rows with sunshine_duration columns: 0
Number of missing rows with wind_speed columns: 0


In [88]:
print(len(df_wd_north))
for factor in factors: 
    print(f"Number of missing rows with {factor} columns: {len(df_wd_north[df_wd_north[factor].isnull()])}")

87672
Number of missing rows with temperature columns: 0
Number of missing rows with snow columns: 201
Number of missing rows with thunder columns: 201
Number of missing rows with sunshine_duration columns: 0
Number of missing rows with wind_speed columns: 0


In [89]:
print(len(df_wd_west))
for factor in factors: 
    print(f"Number of missing rows with {factor} columns: {len(df_wd_west[df_wd_west[factor].isnull()])}")

87672
Number of missing rows with temperature columns: 0
Number of missing rows with snow columns: 20
Number of missing rows with thunder columns: 20
Number of missing rows with sunshine_duration columns: 0
Number of missing rows with wind_speed columns: 0


In [90]:
df_wd_north = df_wd_north[(df_wd_north['sunshine_duration']>=-1.0) & (df_wd_north['sunshine_duration']<=1.0) & 
            ((df_wd_north['snow']==1.0) | (df_wd_north['snow']==0.0)) & 
            ((df_wd_north['thunder']==0.0) | (df_wd_north['thunder']==1.0))]

df_wd_south = df_wd_south[(df_wd_south['sunshine_duration']>=-1.0) & (df_wd_south['sunshine_duration']<=1.0) & 
            ((df_wd_south['snow']==1.0) | (df_wd_south['snow']==0.0)) & 
            ((df_wd_south['thunder']==0.0) | (df_wd_south['thunder']==1.0))]

df_wd_west = df_wd_west[(df_wd_west['sunshine_duration']>=-1.0) & (df_wd_north['sunshine_duration']<=1.0) & 
            ((df_wd_west['snow']==1.0) | (df_wd_west['snow']==0.0)) & 
            ((df_wd_west['thunder']==0.0) | (df_wd_west['thunder']==1.0))]

In [91]:
print(len(df_wd_south))
for factor in factors: 
    print(f"Number of missing rows with {factor} columns: {len(df_wd_south[df_wd_south[factor].isnull()])}")

87672
Number of missing rows with temperature columns: 0
Number of missing rows with snow columns: 0
Number of missing rows with thunder columns: 0
Number of missing rows with sunshine_duration columns: 0
Number of missing rows with wind_speed columns: 0


In [92]:
print(len(df_wd_north))
for factor in factors: 
    print(f"Number of missing rows with {factor} columns: {len(df_wd_north[df_wd_north[factor].isnull()])}")

87471
Number of missing rows with temperature columns: 0
Number of missing rows with snow columns: 0
Number of missing rows with thunder columns: 0
Number of missing rows with sunshine_duration columns: 0
Number of missing rows with wind_speed columns: 0


In [93]:
print(len(df_wd_west))
for factor in factors: 
    print(f"Number of missing rows with {factor} columns: {len(df_wd_west[df_wd_west[factor].isnull()])}")

87451
Number of missing rows with temperature columns: 0
Number of missing rows with snow columns: 0
Number of missing rows with thunder columns: 0
Number of missing rows with sunshine_duration columns: 0
Number of missing rows with wind_speed columns: 0


### Merging tables together

In [94]:
df_wd_north.columns

Index(['code', 'name', 'temperature', 'snow', 'thunder', 'sunshine_duration',
       'wind_speed', 'year', 'month', 'day_of_year', 'week', 'hour'],
      dtype='object')

In [95]:
df_aqd_north.columns

Index(['code', 'name', 'no_x', 'o_3', 'year', 'month', 'day_of_year', 'week',
       'hour', 'station_type'],
      dtype='object')

In [96]:
df_wd_north

,code,name,temperature,snow,thunder,sunshine_duration,wind_speed,year,month,day_of_year,week,hour
datetime,,,,,,,,,,,,
2012-01-01 00:00:00+01:00,280,Eelde,10.2,0.0,0.0,0.0,8.0,2012,1,1,1,0
2012-01-01 01:00:00+01:00,280,Eelde,10.0,0.0,0.0,0.0,6.0,2012,1,1,1,1
2012-01-01 02:00:00+01:00,280,Eelde,10.1,0.0,0.0,0.0,7.0,2012,1,1,1,2
2012-01-01 03:00:00+01:00,280,Eelde,10.4,0.0,0.0,0.0,8.0,2012,1,1,1,3
2012-01-01 04:00:00+01:00,280,Eelde,10.6,0.0,0.0,0.0,7.0,2012,1,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00+01:00,280,Eelde,10.5,0.0,0.0,0.0,6.0,2021,12,365,53,19
2021-12-31 20:00:00+01:00,280,Eelde,10.9,0.0,0.0,0.0,6.0,2021,12,365,53,20
2021-12-31 21:00:00+01:00,280,Eelde,11.3,0.0,0.0,0.0,6.0,2021,12,365,53,21


In [97]:
df_north = df_aqd_north[['code', 'name', 'no_x', 'o_3','station_type']].join(
     df_wd_north[['code', 'name', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed',
                  'year', 'month', 'day_of_year','week', 'hour']], lsuffix= '_aq',how='inner')

df_south = df_aqd_south[['code', 'name', 'no_x', 'o_3','station_type']].join(
     df_wd_south[['code', 'name', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed',
                  'year', 'month', 'day_of_year','week', 'hour']], lsuffix= '_aq',how='inner')

df_west = df_aqd_west[['code', 'name', 'no_x', 'o_3','station_type']].join(
     df_wd_west[['code', 'name', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed',
                  'year', 'month', 'day_of_year','week', 'hour']], lsuffix= '_aq',how='inner')

In [98]:
df_north = df_north[(df_north['o_3']>=0) & (df_north['no_x']>=0)]
df_south = df_south[(df_south['o_3']>=0) & (df_south['no_x']>=0)]
df_west = df_west[(df_west['o_3']>=0) & (df_west['no_x']>=0)]

### Invesitgation of newly created datasets

In [99]:
# NORTH
o3_north = df_north.loc[:, df_north.columns != 'no_x']
o3_north = o3_north[o3_north['o_3'].isnull() != True]  
# Previously we had 172069 rows, however after getting rid of NaN values we get 85549 rows
# Only data from NL10938

nox_north = df_north.loc[:, df_north.columns != 'o_3']
nox_north = nox_north[nox_north['no_x'].isnull() != True] 
# previously we had 172059 rows, however after getting rid of NaN values we get 169805 rows

In [100]:
print(f"Number of rows with o_3 at NL10938: {len(o3_north[o3_north['code_aq']=='NL10938'])}")
print(f"Number of rows with no_x at NL10938: {len(nox_north[nox_north['code_aq']=='NL10938'])}")

Number of rows with o_3 at NL10938: 82561
Number of rows with no_x at NL10938: 82561


In [101]:
# SOUTH
o3_south = df_south.loc[:, df_south.columns != 'no_x']
o3_south = o3_south[o3_south['o_3'].isnull() != True]
# Previously we had 385370 rows, however after getting rid of NaN values we get 170506 rows
# Only data from 'NL10133', 'NL10137', 'NL10138'

nox_south = df_south.loc[:, df_south.columns != 'o_3']
nox_south = nox_south[nox_south['no_x'].isnull() != True]
# Previously we had 385370 rows, however after getting rid of NaN values we get 381784

In [102]:
print(f"Number of rows with o_3 at NL10133: {len(o3_south[o3_south['code_aq']=='NL10133'])}")
print(f"Number of rows with no_x at NL10133: {len(nox_south[nox_south['code_aq']=='NL10133'])}")
print()
print(f"Number of rows with o_3 at NL10137: {len(o3_south[o3_south['code_aq']=='NL10137'])}")
print(f"Number of rows with no_x at NL10137: {len(nox_south[nox_south['code_aq']=='NL10137'])}")
print()
print(f"Number of rows with o_3 at NL10138: {len(o3_south[o3_south['code_aq']=='NL10138'])}")
print(f"Number of rows with no_x at NL10138: {len(nox_south[nox_south['code_aq']=='NL10138'])}")

Number of rows with o_3 at NL10133: 80204
Number of rows with no_x at NL10133: 80204

Number of rows with o_3 at NL10137: 17278
Number of rows with no_x at NL10137: 17278

Number of rows with o_3 at NL10138: 67288
Number of rows with no_x at NL10138: 67288


In [103]:
# WEST
o3_west = df_west.loc[:, df_west.columns != 'no_x']
o3_west = o3_west[o3_west['o_3'].isnull() != True]
# Previously we had 96015 rows, however after getting rid of NaN values we get 82348 rows
# Data only from 'NL10318'

nox_west = df_west.loc[:, df_west.columns != 'o_3']
nox_west = nox_west[nox_west['no_x'].isnull() != True]
# Previously we had 96015 rows, however after getting rid of NaN values we get 93761

In [104]:
print(f"Number of rows with o_3 at NL10318: {len(o3_west[o3_west['code_aq']=='NL10318'])}")
print(f"Number of rows with no_x at NL10318: {len(nox_west[nox_west['code_aq']=='NL10318'])}")

Number of rows with o_3 at NL10318: 79460
Number of rows with no_x at NL10318: 79460


### Division of data on "Cold" and "Warm" months MEAN

In [105]:
# NORTH

mean_o3 = o3_north.groupby(['year', 'month', 'week'])['o_3', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].mean()
mean_nox = nox_north.groupby(['year', 'month', 'week'])['no_x', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].mean()

north_warm_o3 = mean_o3[mean_o3['temperature']>=10]
north_cold_o3 = mean_o3[mean_o3['temperature']<10]

north_warm_nox = mean_nox[mean_nox['temperature']>=10]
north_cold_nox = mean_nox[mean_nox['temperature']<10]

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_90479/2557042681.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_o3 = o3_north.groupby(['year', 'month', 'week'])['o_3', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].mean()
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_90479/2557042681.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_nox = nox_north.groupby(['year', 'month', 'week'])['no_x', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].mean()


In [106]:
# SOUTH

mean_o3 = o3_south.groupby(['year', 'month', 'week'])['o_3', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].mean()
mean_nox = nox_south.groupby(['year', 'month', 'week'])['no_x', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].mean()

south_warm_o3 = mean_o3[mean_o3['temperature']>=10]
south_cold_o3 = mean_o3[mean_o3['temperature']<10]

south_warm_nox = mean_nox[mean_nox['temperature']>=10]
south_cold_nox = mean_nox[mean_nox['temperature']<10]

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_90479/566323835.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_o3 = o3_south.groupby(['year', 'month', 'week'])['o_3', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].mean()
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_90479/566323835.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_nox = nox_south.groupby(['year', 'month', 'week'])['no_x', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].mean()


In [107]:
# WEST

mean_o3 = o3_south.groupby(['year', 'month', 'week'])['o_3', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].mean()
mean_nox = nox_south.groupby(['year', 'month', 'week'])['no_x', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].mean()

west_warm_o3 = mean_o3[mean_o3['temperature']>=10]
west_cold_o3 = mean_o3[mean_o3['temperature']<10]

west_warm_nox = mean_nox[mean_nox['temperature']>=10]
west_cold_nox = mean_nox[mean_nox['temperature']<10]

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_90479/398927122.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_o3 = o3_south.groupby(['year', 'month', 'week'])['o_3', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].mean()
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_90479/398927122.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_nox = nox_south.groupby(['year', 'month', 'week'])['no_x', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].mean()


### Division of data on "Cold" and "Warm" months MEAN

In [108]:
# NORTH

mean_o3 = o3_north.groupby(['year', 'month', 'week'])['o_3', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].median()
mean_nox = nox_north.groupby(['year', 'month', 'week'])['no_x', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].median()

north_warm_o3 = mean_o3[mean_o3['temperature']>=10]
north_cold_o3 = mean_o3[mean_o3['temperature']<10]

north_warm_nox = mean_nox[mean_nox['temperature']>=10]
north_cold_nox = mean_nox[mean_nox['temperature']<10]

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_90479/3318209221.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_o3 = o3_north.groupby(['year', 'month', 'week'])['o_3', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].median()
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_90479/3318209221.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_nox = nox_north.groupby(['year', 'month', 'week'])['no_x', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].median()


In [109]:
# SOUTH

mean_o3 = o3_south.groupby(['year', 'month', 'week'])['o_3', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].median()
mean_nox = nox_south.groupby(['year', 'month', 'week'])['no_x', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].median()

south_warm_o3 = mean_o3[mean_o3['temperature']>=10]
south_cold_o3 = mean_o3[mean_o3['temperature']<10]

south_warm_nox = mean_nox[mean_nox['temperature']>=10]
south_cold_nox = mean_nox[mean_nox['temperature']<10]

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_90479/295881656.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_o3 = o3_south.groupby(['year', 'month', 'week'])['o_3', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].median()
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_90479/295881656.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_nox = nox_south.groupby(['year', 'month', 'week'])['no_x', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].median()


In [110]:
# WEST

mean_o3 = o3_south.groupby(['year', 'month', 'week'])['o_3', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].median()
mean_nox = nox_south.groupby(['year', 'month', 'week'])['no_x', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].median()

west_warm_o3 = mean_o3[mean_o3['temperature']>=10]
west_cold_o3 = mean_o3[mean_o3['temperature']<10]

west_warm_nox = mean_nox[mean_nox['temperature']>=10]
west_cold_nox = mean_nox[mean_nox['temperature']<10]

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_90479/468469341.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_o3 = o3_south.groupby(['year', 'month', 'week'])['o_3', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].median()
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_90479/468469341.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_nox = nox_south.groupby(['year', 'month', 'week'])['no_x', 'temperature', 'snow', 'thunder', 'sunshine_duration', 'wind_speed'].median()


### Getting rid of NaN values in the dataframe

In [111]:
# dick = df_aqd_north[['code', 'name', 'no', 'Station type']][['no']].dropna(axis = 0)
# dick[['no']].describe()

In [112]:
df_aqd_north['code'].unique()

array(['NL10937', 'NL10938'], dtype=object)

In [113]:
df_aqd_north['Station type'].unique()

KeyError: 'Station type'

In [ ]:
df_aqd_north[df_aqd_north['Station type'] == 'street'].plot(kind = 'scatter', x = 'week', y = 'no_x');

In [ ]:
df_aqd_north[df_aqd_north['Station type'] == 'city'].plot(kind = 'scatter', x = 'week', y = 'no_x');

In [ ]:
fig, ax = plt.subplots(nrows= 2, ncols = 1, figsize = (15, 5))
sns.boxplot(data = df_aqd_north[df_aqd_north['Station type'] == 'street'], x = 'week', y= 'no',ax=ax[0])
sns.boxplot(data = df_aqd_north[df_aqd_north['Station type'] == 'street'], x = 'week', y= 'no_median',ax=ax[1]);
# ax.set_ylim(-10, 250);

In [ ]:
df_aqd_north

In [ ]:
abc = df_aqd_north[df_aqd_north['Station type'] == 'city']
abc_year = abc.groupby(['year', 'day_of_year']).agg('mean')
abc_day = abc_year.groupby(['day_of_year']).agg('mean')
abc_day['no'].plot()
abc_day['no_median'].plot()


In [ ]:
df_hui = df_aqd_north[df_aqd_north['Station type'] == 'street'].copy()
new_df = df_hui.describe()[['no']]

In [ ]:
iqr = new_df.loc['75%']-new_df.loc['25%']
upper_bound = 1.5*iqr + new_df.loc['75%']
lower_bound = 1.5*iqr - new_df.loc['25%']

In [ ]:
df_hui['no_outliers_no'] = (hui[['no']]>upper_bound) | (hui[['no']]< lower_bound)

In [ ]:
fig, ax = plt.subplots(nrows= 1, ncols = 1, figsize = (15, 5))
sns.boxplot(data = df_hui[(df_hui['Station type'] == 'street') & (df_hui['no_outliers_no']==False)], x = 'week', y= 'no', ax = ax);



In [ ]:
abc = df_hui[(df_hui['Station type'] == 'street') & (df_hui['no_outliers_no']==False)]
abc_year = abc.groupby(['year', 'day_of_year']).agg('mean')
abc_day = abc_year.groupby(['day_of_year']).agg('mean')
abc_day[['no', 'no_median']].plot()

In [ ]:
df_aqd_north[['no_x']].info()

## Part 3. Hypothesis testing and interpretation

In [ ]:
# Use this cell as you like, and add more cells as needed.

## Part 4. Pitching results

In [ ]:
# This section is only for generating figures if you need it. You may leave it empty.

# Feedback

Please fill in this questionaire to help us improve this course for the next year. Your feedback will be anonymized and will not affect your grade in any way!

### How many hours did you spend on these Exercises?

Assign a number to `feedback_time`.

In [ ]:
#// BEGIN_FEEDBACK [Feedback_1] (0 point)

#// END_FEEDBACK [Feedback_1] (0 point)

import numbers
assert isinstance(feedback_time, numbers.Number), "Please assign a number to feedback_time"
feedback_time

### How difficult did you find these Exercises?

Assign an integer to `feedback_difficulty`, on a scale 0 - 10, with 0 being very easy, 5 being just right, and 10 being very difficult.

In [ ]:
#// BEGIN_FEEDBACK [Feedback_2] (0 point)

#// END_FEEDBACK [Feedback_2] (0 point)

import numbers
assert isinstance(feedback_difficulty, numbers.Number), "Please assign a number to feedback_difficulty"
feedback_difficulty

### (Optional) What did you like?

Assign a string to `feedback_like`.

In [ ]:
#// BEGIN_FEEDBACK [Feedback_3] (0 point)

#// END_FEEDBACK [Feedback_3] (0 point)

### (Optional) What can be improved?

Assign a string to `feedback_improve`. Please be specific, so that we can act on your feedback. For example, mention the specific exercises and what was unclear.

In [ ]:
#// BEGIN_FEEDBACK [Feedback_4] (0 point)

#// END_FEEDBACK [Feedback_4] (0 point)




## How to Submit Your Work

1. **Before submitting**, you must run your notebook by doing **Kernel > Restart & Run All**.  
   Make sure that your notebook runs without errors **in linear order**.
1. Remember to rename the notebook, replacing `...-template.ipynb` with `...-yourIDnr.ipynb`, where `yourIDnr` is your TU/e identification number.
1. Submit the executed notebook with your work
   for the appropriate assignment in **Canvas**.
1. In the **Momotor** tab in Canvas,
  you can select that assignment again to find some feedback on your submitted work.
  If there are any problems reported by _Momotor_,
  then you need to fix those,
  and **resubmit the fixed notebook**.

In case of a high workload on our server
(because many students submit close to the deadline),
it may take longer to receive the feedback.




---

In [ ]:
# List all defined names
%whos

---

# (End of Notebook) <span class="tocSkip"></span>

&copy; 2017-2023 - **TU/e** - Eindhoven University of Technology